## 115. Distinct Subsequences
- Description:
  <blockquote>
    Given two strings s and t, returnthe number of distinctsubsequencesofswhich equalst.
     
    The test cases are generated so that the answer fits on a 32-bit signed integer.
     
    **Example 1:**
    **Input:** s = "rabbbit", t = "rabbit"
    **Output:** 3
    **Explanation:**
    As shown below, there are 3 ways you can generate "rabbit" from s.
    `**rabb**b**it**`
    `**ra**b**bbit**`
    `**rab**b**bit**`
     
    **Example 2:**
    **Input:** s = "babgbag", t = "bag"
    **Output:** 5
    **Explanation:**
    As shown below, there are 5 ways you can generate "bag" from s.
    `**ba**b**g**bag`
    `**ba**bgba**g**`
    `**b**abgb**ag**`
    `ba**b**gb**ag**`
    `babg**bag**`
     
    **Constraints:**
     
    - `1 <= s.length, t.length <= 1000`
    - `s` and `t` consist of English letters.
  </blockquote>

- URL: [Problem_URL](https://leetcode.com/problems/distinct-subsequences/description/)

- Topics: Recursion with Memoization, Dynamic Programming

- Difficulty: Medium / Hard

- Resources: example_resource_URL

### Solution 1, Recursion + Memoization
Solution description

Where M and N represent the lengths of the two strings. 

- Time Complexity: O(M×N)
  - The number of unique recursive calls is defined by the two state variables that we have. Potentially, we can make O(M×N) calls where M and N represent the lengths of the two strings. Thus, the time complexity for this solution would be O(M×N).
- Space Complexity: O(M×N)
  - size of that dictionary would also be controlled by the total possible combinations of i and j which turns out to be O(M×N) as well

In [ ]:
class Solution:
    def numDistinct(self, s: str, t: str) -> int:
        memo = {}
        s_len, t_len = len(s), len(t)

        # The number of distinct ways to form t[j:] (substring of t from index j to end) as a subsequence of s[i:] (substring of s from index i to end).
        # It counts how many distinct subsequences of the remaining part of s (starting at i) match the remaining part of t (starting at j).
        def uniqueSubsequences(i: int, j: int) -> int:
            # Base case
            # Early termination check s_len - i < t_len - j: if remaining characters in s are fewer than the remaining characters needed to match in t.
            if i == s_len or j == t_len or s_len - i < t_len - j:
                return 1 if j == t_len else 0

            # Check if the result is already cached
            if (i, j) in memo:
                return memo[i, j]

            # Case 1: when we skip the character in "s"
            # if the characters don't match, we have no choice but to skip the character in "s"
            # if the characters match, we can still choose to skip it
            ans = uniqueSubsequences(i + 1, j)

            # Case 2: when the characters match, we can use it to match
            if s[i] == t[j]:
                ans += uniqueSubsequences(i + 1, j + 1)

            # Cache the answer and return
            memo[i, j] = ans
            return ans

        return uniqueSubsequences(0, 0)

### Solution 2, 2D Dynamic Programming


       j: 0    1    2
          ''  'b'  'bg'
i:
0  ''     1    0    0
1  'b'    1    1    0
2  'ba'   1    1    0
3  'bag'  1    1    1

Where M and N represent the lengths of the two strings. 
- Time Complexity: O(M×N)
  - The time complexity is much more clear in this approach since we have two for loops with clearly defined executions. The outer loop runs for M+1 iterations while the inner loop runs for N+1 iterations. So, combined together we have a time complexity of O(M×N).
- Space Complexity: O(M×N)
  - which is occupied by the 2D dp array that we create. 

In [ ]:
class Solution:
    def numDistinct(self, s: str, t: str) -> int:
        s_len, t_len = len(s), len(t)
        
        # Create DP table: dp[i][j] = ways to form t[0:j] from s[0:i]
        # Extra row and column for base cases
        dp = [[0] * (t_len + 1) for _ in range(s_len + 1)]
        
        # Base case: empty t can be formed in 1 way from any prefix of s
        for i in range(s_len + 1):
            dp[i][0] = 1
        
        # Fill table forwards
        for i in range(1, s_len + 1):
            for j in range(1, t_len + 1):
                # Early termination: not enough characters left in s to match t
                if s_len - i < t_len - j:
                    dp[i][j] = 0
                    continue
                
                # Don't use s[i-1], skip it
                dp[i][j] = dp[i - 1][j]
                
                # If characters match, add ways from using s[i-1]
                if s[i - 1] == t[j - 1]:
                    dp[i][j] += dp[i - 1][j - 1]
        
        return dp[s_len][t_len]

### Solution 3, Space optimized Dynamic Programming

Solution description
- Time Complexity: O(M*N)
- Space Complexity: O(N)
  - since we are using a single array which is the size of the string T. This is a major size reduction over the previous solution and this is a much more elegant solution than the initial recursive solution we saw earlier on.

In [ ]:
class Solution:
    def numDistinct(self, s: str, t: str) -> int:
        s_len, t_len = len(s), len(t)
        
        # 1D DP array: dp[j] = ways to form t[0:j] from current prefix of s
        dp = [0] * (t_len + 1)
        dp[0] = 1  # Base case: empty t can be formed in 1 way
        
        # Iterate through each character in s
        for i in range(1, s_len + 1):
            # Iterate through t BACKWARDS to avoid overwriting needed values
            # dp[j] initially holds dp[i-1][j] (previous row, don't match s[i-1])
            # dp[j-1] still holds dp[i-1][j-1] (not yet updated)
            for j in range(t_len, 0, -1):
                # Early termination: not enough characters left
                if s_len - i < t_len - j:
                    dp[j] = 0
                    continue
                
                # If characters match, add ways from matching
                if s[i - 1] == t[j - 1]:
                    dp[j] += dp[j - 1]
                # If not match, dp[j] stays the same (skip s[i-1])
        
        return dp[t_len]